In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import holodeck as holo
from holodeck.constants import GYR, YR, PC, MSOL

In [ ]:
mmbulge = holo.relations.MMBulge_KH2013()
gsmf = holo.sams.comps.GSMF_Double_Schechter()
gmr = holo.sams.comps.GMR_Illustris()

sam = holo.sams.sam.Semi_Analytic_Model(gsmf=gsmf, gmr=gmr, mmbulge=mmbulge)
print(sam)

In [ ]:
HARD_TIME = 2 * GYR
hard = holo.hardening.Fixed_Time_2PL_SAM()